# Optimizing Model Paramters
---

* [Link](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)
* [Backpropagation](https://www.youtube.com/watch?v=tIeHLnjs5U8)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [9]:
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

device = "cuda" if torch.cuda.is_available() else "cpu"

model = NeuralNetwork().to(device)

# Hyperparameters
---

* **Number of Epochs**
* **Batch Size**
* **Learning Rate**

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimization Loop
---

Each epoch consists of two main parts:
* **The Train Loop**
* **The Validation/Test Loop**

# Loss Function
---

Common loss functions:
* [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error) for regression tasks
* [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification
* [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combines the above two

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
---

[Different optimizers](https://pytorch.org/docs/stable/optim.html) such as SGD, ADAM and RMSProp

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Full Implementation
---

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 2.172617 [    0/60000]
loss: 2.152074 [ 6400/60000]
loss: 2.100726 [12800/60000]
loss: 2.108446 [19200/60000]
loss: 2.043392 [25600/60000]
loss: 2.003994 [32000/60000]
loss: 2.017648 [38400/60000]
loss: 1.952383 [44800/60000]
loss: 1.959922 [51200/60000]
loss: 1.866995 [57600/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.874407 

Epoch 2
--------------------
loss: 1.921453 [    0/60000]
loss: 1.873162 [ 6400/60000]
loss: 1.765620 [12800/60000]
loss: 1.799228 [19200/60000]
loss: 1.681347 [25600/60000]
loss: 1.647196 [32000/60000]
loss: 1.659986 [38400/60000]
loss: 1.576013 [44800/60000]
loss: 1.597757 [51200/60000]
loss: 1.486234 [57600/60000]
Test Error: 
 Accuracy: 60.4%, Avg loss: 1.508141 

Epoch 3
--------------------
loss: 1.585629 [    0/60000]
loss: 1.536871 [ 6400/60000]
loss: 1.396313 [12800/60000]
loss: 1.462641 [19200/60000]
loss: 1.341232 [25600/60000]
loss: 1.342085 [32000/60000]
loss: 1.355662 [38400/60000]
loss: 1.290207 [44800/6000

# Further Reading
---

* [Hyperparameter Tuning Tutorial](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)
* [Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)
* [torch.optim](https://pytorch.org/docs/stable/optim.html)
* [Warmstart Training a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)
* [Backpropagation](https://www.youtube.com/watch?v=tIeHLnjs5U8)